In [202]:
import requests
import re
import urllib.parse
import json
import io


class Project:
    def __init__(self, id: int, server=None):
        if not id:
            raise Exception('Invalid project id')

        self.id = id
        self.server: Server = server

    def __repr__(self):
        return f"Project(id={self.id}; server={self.server.url})"

    def get_models(self):
        return self.server.get(
            'command/core/get-models',
            query={'project': self.id}
        ).json()

    def apply_operations(self, operations_json: list):
        return self.server.post(
            'command/core/apply-operations', 
            query={'project': self.id},  # the docs say this is where it belongs, but reality says it's in the data
            data={
                'project': self.id, 
                'operations': json.dumps(operations_json)
            }
        ).text


class Server:
    def __init__(self, url: str = "http://localhost:3333"):
        self.url = url
        self.token = None

    @property
    def is_connected(self):
        return self.token is not None

    def get_csrf_token(self) -> str:
        response = requests.get(f'{self.url}/command/core/get-csrf-token')
        j = response.json()
        if 'token' not in j:
            raise Exception("Invalid response")

        return j['token']

    def connect(self):
        self.token = self.get_csrf_token()
        return self

    def disconnect(self):
        self.token = None
        return self

    def get(self, path, query=None, headers=None, *args, **kwargs):
        if query is None:
            query = {}
        q = urllib.parse.urlencode({**query, 'csrf_token': self.token})

        if headers is None:
            headers = {}

        headers = {
            'Accept': 'application/xml,*/*;0.8',
            **headers
        }

        r = requests.get(f"{self.url}/{path}?{q}", *args, headers=headers, **kwargs)

        if r.status_code >= 400 and r.status_code < 500:
            raise Exception(f"Invalid Request: Status Code {r.status_code}")
        if r.status_code >= 500 and r.status_code < 600:  
            raise Exception(f"Internal Server Error: Status Code {r.status_code}")

        return r

    def post(self, path, query=None, headers=None, *args, **kwargs):
        if query is None:
            query = {}
        q = urllib.parse.urlencode({**query, 'csrf_token': self.token})

        if headers is None:
            headers = {}

        headers = {
            # 'Accept': 'application/xml,*/*;0.8',
            'Accept': '*/*',
            **headers
        }
        r = requests.post(f"{self.url}/{path}?{q}", *args, headers=headers, **kwargs)

        if r.status_code >= 400 and r.status_code < 500:
            print(r.text)
            raise Exception(f"Invalid Request: Status Code {r.status_code}")
        if r.status_code >= 500 and r.status_code < 600:
            print(r.text)
            raise Exception(f"Internal Server Error: Status Code {r.status_code}")

        return r

    def create_project_from_file(self, file_path: str) -> Project:
        if not self.token:
            self.connect()

        path = f'command/core/create-project-from-upload'
        files = {'project-file': open(file_path, 'rb')}
        data = {
            'project-name': 'New Project',
            'format': 'test/line-based/*sv',
            'options': {
                # "encoding":"UTF-8",
                # "separator":",",
                # "ignoreLines":-1,
                # "headerLines":1,
                # "skipDataLines":0,
                # "limit":-1,
                # "storeBlankRows": True,
                # "guessCellValueTypes": True,
                # "processQuotes": True,
                # "quoteCharacter": "\"",
                # "storeBlankCellsAsNulls": True,
                # "includeFileSources": False,
                # "includeArchiveFileName": False,
                # "trimStrings": False,
                # "disableAutoPreview": False,
                # "projectName":"Menu csv",
                # "projectTags":[]

            }
        }
        headers = {
            'Accept': 'application/xml,*/*;0.8',
        }
        r = self.post(path, data=data, files=files, headers=headers)

        if not 'project=' in r.url:
            print(r.text)
            raise Exception('Project Creation Failure')

        v = re.search(r'^.+project=(\d+).*$', r.url, re.I | re.S)

        return Project(int(v.group(1)), server=self)

    def __repr__(self):
        return f"Server(url={self.url}; token={self.token})"


project = Server().create_project_from_file('data/Menu.csv')
print(project)

Project(id=1905751276947; server=http://localhost:3333)


In [203]:
project.get_models()

{'columnModel': {'columns': [{'cellIndex': 0,
    'originalName': 'id',
    'name': 'id'},
   {'cellIndex': 1, 'originalName': 'name', 'name': 'name'},
   {'cellIndex': 2, 'originalName': 'sponsor', 'name': 'sponsor'},
   {'cellIndex': 3, 'originalName': 'event', 'name': 'event'},
   {'cellIndex': 4, 'originalName': 'venue', 'name': 'venue'},
   {'cellIndex': 5, 'originalName': 'place', 'name': 'place'},
   {'cellIndex': 6,
    'originalName': 'physical_description',
    'name': 'physical_description'},
   {'cellIndex': 7, 'originalName': 'occasion', 'name': 'occasion'},
   {'cellIndex': 8, 'originalName': 'notes', 'name': 'notes'},
   {'cellIndex': 9, 'originalName': 'call_number', 'name': 'call_number'},
   {'cellIndex': 10, 'originalName': 'keywords', 'name': 'keywords'},
   {'cellIndex': 11, 'originalName': 'language', 'name': 'language'},
   {'cellIndex': 12, 'originalName': 'date', 'name': 'date'},
   {'cellIndex': 13, 'originalName': 'location', 'name': 'location'},
   {'cellInd

In [204]:
op = [
    {
        "op": "core/column-addition",
        "engineConfig": {
            "facets": [],
            "mode": "row-based"
        },
        "baseColumnName": "sponsor",
        "expression": "grel:value.type()",
        "onError": "set-to-blank",
        "newColumnName": "sponsor_type",
        "columnInsertIndex": 3,
        "description": "Create column sponsor_type at index 3 based on column sponsor using expression grel:value.type()"
    }
]

project.apply_operations(op)

'{ "code" : "ok" }'

In [205]:
project.get_models()

{'columnModel': {'columns': [{'cellIndex': 0,
    'originalName': 'id',
    'name': 'id'},
   {'cellIndex': 1, 'originalName': 'name', 'name': 'name'},
   {'cellIndex': 2, 'originalName': 'sponsor', 'name': 'sponsor'},
   {'cellIndex': 20, 'originalName': 'sponsor_type', 'name': 'sponsor_type'},
   {'cellIndex': 3, 'originalName': 'event', 'name': 'event'},
   {'cellIndex': 4, 'originalName': 'venue', 'name': 'venue'},
   {'cellIndex': 5, 'originalName': 'place', 'name': 'place'},
   {'cellIndex': 6,
    'originalName': 'physical_description',
    'name': 'physical_description'},
   {'cellIndex': 7, 'originalName': 'occasion', 'name': 'occasion'},
   {'cellIndex': 8, 'originalName': 'notes', 'name': 'notes'},
   {'cellIndex': 9, 'originalName': 'call_number', 'name': 'call_number'},
   {'cellIndex': 10, 'originalName': 'keywords', 'name': 'keywords'},
   {'cellIndex': 11, 'originalName': 'language', 'name': 'language'},
   {'cellIndex': 12, 'originalName': 'date', 'name': 'date'},
   {